# Method comparison
In this notebook, three different methods of doing core-loss quantification will be compared to each other on a simulated dataset.
1. **Conventional**: Conventional background removal with a power-law
2. **Power-law Model**: Model-based fitting with power-law background
3. **Linear Model**: Model-based fitting with sum of power-laws background

In [1]:
%matplotlib qt

In [2]:
from pyEELSMODEL.operators.simulator.coreloss_simulator import CoreLossSimulator
from pyEELSMODEL.operators.quantification.elemental_quantification import ElementalQuantification
from pyEELSMODEL.operators.backgroundremoval import BackgroundRemoval

from pyEELSMODEL.core.multispectrum import MultiSpectrumshape, MultiSpectrum
import pyEELSMODEL.api as em

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

## Simulation core-loss
In this section , the core-loss signal will be simulated. *This step is in general not needed since we get the experimental data instead of simulating it.*


In [3]:
def make_circular_mask(xco, yco, Rin, Rout, shape):
    XX, YY = np.meshgrid(np.arange(shape[1])-yco, np.arange(shape[0])-xco)
    
    R = np.sqrt(XX**2+YY**2)
    
    mask = np.zeros(shape)
    boolean = (R>=Rin) & (R<Rout)
    mask[boolean] = 1
    return mask

def make_rectangular_mask(xco, yco, width,height, shape):
    mask = np.zeros(shape)
    mask[xco:xco+width, yco:yco+height] = 1
    return mask

#### Elemental abundance maps
Define the elements and the eges used. Add the atomic numbers since they will be used to have some estimate on the free mean path. \
Different masks are defined to showcase an elemental abundance at each position

In [4]:
elements = ['C', 'N', 'O', 'Fe']
edges = ['K', 'K', 'K', 'L']
Zs = [6, 7, 8, 26] #atomic weights

#scan size elemental map
xsize = 128
ysize = 128
maps = np.zeros((len(elements),xsize,ysize))

#masks which define different regions. 
mask0 =make_rectangular_mask(5, 5, 20, 20, (xsize,ysize))
mask1 =  make_rectangular_mask(90, 90, 20, 30, (xsize,ysize))
mask2 = make_circular_mask(xsize//2, ysize//2, 20, 30, (xsize,ysize))
mask3 = make_circular_mask(xsize//2, ysize//2, 0, 20, (xsize,ysize))

#attribute different elemental values for different masks
maps[0] = 1  #carbon elemental map
maps[1] = 2*mask0 + mask1 #nitrogen elemental map
maps[2] = mask2 #oxygen elemental map
maps[3] = mask3+0.5*mask2 #iron elemental map

In [5]:
#shows the real truth elemental abundance
fig, ax = plt.subplots(1,len(elements))
for ii in range(maps.shape[0]):
    ax[ii].imshow(maps[ii], cmap='gray')
    ax[ii].set_title(elements[ii])

#### t/&lambda;
Calculate some measure of inelastic mean free path to include this into the simulated data. Since a change in inelastic mean free path modfies the multiple scattering which then changes the shape of background and core loss edge

In [6]:
adf = np.zeros((xsize, ysize))
tlambda_map = np.zeros_like(adf)
for ii in range(maps.shape[0]):
    adf += (Zs[ii]*maps[ii])**2
    tlambda_map += Zs[ii]*maps[ii]
    
tlambda_map = tlambda_map/tlambda_map.max() #maximum t/lambda is 1 for this sample

In [7]:
#shows adf and mean-free path figure
fig, ax = plt.subplots(1,2)
ax[0].imshow(adf, cmap='gray')
ax[0].set_title(r'ADF like contrast')
ax[1].imshow(tlambda_map, cmap='gray')
ax[1].set_title('mean free path')

Text(0.5, 1.0, 'mean free path')

#### Simulation part
CoreLossSimulator is an object which simulates a core loss multispectrum from the given elements, maps, t/&lambda; and the settings.\
Here is a small description on how the multispectrum gets simulated:
1. Calculate a low loss which contains a zero-loss (Gaussian) with a FWHM (1eV) and the low loss. The low loss only uses the bulk plasmons, these are calculated using the plasmon energy (22 eV) and t/&lambda;. In this simulation the only variying parameter in the low loss is the t/&lambda; . This is done to showcase the influence of multiple scattering on the core-loss spectra.
2. The background is calculated by starting with a powerlaw (A E<sup>-3</sup>) which gets convolved with the low loss at each probe position. This modifies the shape of the background. 
3. The core loss edges are calculated using the E0, alpha, beta and GOS tables available. The calculated shape then gets convolved with the low loss to thake the multiple scattering into account. The GOS tables used are the one calculated by [Zhang Zezhong](https://zenodo.org/records/7729585). Note that no fine structure is added to the edges which is something that is always present in experimental EEL spectra. 
4. Poisson noise is added to the core-loss.

In [16]:
E0 = 300e3 #acceleration voltage [V]
alpha = 1e-9 #convergence angle [rad]
beta = 20e-3 #collection angle [rad]

dispersion = 0.5 #[eV]
offset = 200 #[eV]
size = 2048 #number of pixel used in simulation [eV]

settings = (E0, alpha, beta)
msh = em.MultiSpectrumshape(dispersion, offset, size, xsize, ysize)

In [9]:
sim = em.CoreLossSimulator(msh, elements, edges, maps, tlambda_map, settings)
sim.use_shift = False #no shifts are added
sim.simulate_multispectrum()

16384it [00:13, 1202.08it/s]
16384it [00:36, 443.36it/s]

Multispectrum is simulated


In [17]:
#redefine the multispectra for easy use
hl = sim.multispectrum
ll = sim.ll

## Quantification Part
In this section, the three methods are used to obtain quantitative results on EELS map. 


#### Conventional
The conventional method selects a pre-region from each edge. These regions should be selected by the user. These pre regions are used to fit a power-law function ($AE^{-r}$). Since this is a non-linear function, proper starting values should be provided. These starting values are estimated by performing a linear fit on:\
$$\log S(E) = \log A - r \log E,$$ 
where S(E) is the measured intensity, E the energy axis [eV], A the amplitude of the power-law and r the power of the power-law.
These values will be given as starting parameters for the non-linear fit. Note, other methods can be used to estimate the starting values of the power-law but are not used at this point. \
The next step is to integrate the signal under the remaining edge ($I_m$). The interval regions [$E_1$, $E_2$] over which to integrate are selected by the user could be described by following function.
$$I_m =   \int_{E_1}^{E_2} S(E) - bg(E) dE$$
To get an estimate on the real abundance, this value should be compared to the theoretical cross section. This cross section is calculated by using the atomic GOS ($\sigma_A$) and the experimental parameters; acceleration voltage, convergence angle, collection angle and low loss (ZLP). The expected intensity ($I_e$) is then calculated by integrating the theoretical edge over the same interval.
$$I_e = \int_{E_1}^{E_2} ZLP(E) \ast \sigma_A(E, E_0, \alpha, \beta) dE  $$
The resulting abundance is the ratio of measured intensity over expected intensity $I_m/I_e$.


In [11]:
signal_ranges = [[220,280],[350,395],[440,520],[600,700]] #user-selected
int_wins= [[285,350],[400,475],[525,600], [700,800]] #user-selected
elements = ['C', 'N', 'O', 'Fe'] #the sequence of elements chosen in signal_ranges
E0=settings[0]
alpha=settings[1]
beta=settings[2]
int_maps = np.zeros_like(maps)
for ii in range(len(signal_ranges)):
    back = BackgroundRemoval(hl, signal_ranges[ii])
    rem = back.calculate_multi()
    int_maps[ii] = back.quantify_from_edge(int_wins[ii], elements[ii], edges[ii],
                                           E0, alpha, beta, ll=ll)

True


16384it [00:02, 6751.95it/s]
16384it [00:33, 481.93it/s]
16384it [00:01, 14100.49it/s]
16384it [00:08, 1832.40it/s]


True


16384it [00:02, 6192.05it/s]
16384it [00:40, 406.37it/s]
16384it [00:01, 13454.15it/s]
16384it [00:09, 1798.89it/s]


True


16384it [00:02, 6731.93it/s]
16384it [00:41, 391.48it/s]
16384it [00:01, 13542.57it/s]
16384it [00:09, 1819.19it/s]


True


16384it [00:02, 6675.14it/s]
16384it [00:52, 312.08it/s]
16384it [00:01, 14260.09it/s]
16384it [00:09, 1709.24it/s]


In [12]:
fig, ax = plt.subplots(1, int_maps.shape[0])
for ii in range(int_maps.shape[0]):
    ax[ii].imshow(int_maps[ii])
    ax[ii].set_title(elements[ii])

#### Power-law Model
The model-based approach where the power-law is used as background function. The atomic cross sections are calculated using the GOS tables calculated by [Zhang Zezhong](https://zenodo.org/records/7729585) Multiple scattering is taken into account by adding the low-loss multispectrum. Here, the ElementalQuantification class is used for the quantification workflow. If one is interested in how to do the workflow by itself, then I would advice to have a look at the Coreloss Example notebook or have a look at the source code used in the ElementalQuantification class. 

In [13]:
elements = ['C', 'N', 'O', 'Fe']
edges = ['K', 'K', 'K', 'L']

E0 = 300e3 #acceleration voltage [V]
alpha = 1e-9 #convergence angle [rad]
beta = 20e-3 #collection angle [rad]
settings = (E0, alpha, beta)

The non-linear fitting procedure is rather slow (20 it/s) on this pc. Hence the computation takes some time +-15 min. 

In [14]:
quantpw = ElementalQuantification(hl, elements, edges, settings, ll=ll)
quantpw.background_model = 'powerlaw' #attribute to indicate the background model
quantpw.do_procedure()

dataset is already aligned


cannot use analytical gradients since a convolutor is inside model
16384it [12:20, 22.12it/s]
16384it [00:07, 2158.95it/s]


#### Linear Model
The model-based approach where background is estimated as the sum of power-laws. In this work, the exact background function is following:
$$ bg(E) = A_0 E^{-1} + A_1 E^{-2.33} + A_2 E^{-3.66} + A_3 E^{-5}, $$
The values of the powers are taken from [Van den Broek et al](https://doi.org/10.1016/j.ultramic.2023.113830). This choice of background reduces the optimization problem to a linear one. This has the advantage of being faster, non-iterative and no starting values are needed. Moreover, as is shown in [Van den Broek et al](https://doi.org/10.1016/j.ultramic.2023.113830), this background function is a better description of a real background signal in EELS compared to a power-law especially over large energy windows. The atomic cross sections and low loss is the same as for the **Power-law Method**.

In [18]:
quant = ElementalQuantification(hl, elements, edges, settings, ll=ll)
quant.n_bgterms = 4 #attribute to indicate the number of terms used in the linear background model
quant.linear_fitter_method = 'ols' #the optimization method, ols (ordinary least squares) non-negative is also available (nnls)
quant.do_procedure()

dataset is already aligned


cannot use analytical gradients since a convolutor is inside model
16384it [00:32, 502.75it/s]
16384it [00:11, 1430.99it/s]


## Comparison 
The last section shows some comparisons between the different methods. The first part compares the resulting fits of **Power-law Method** and **Linear Method** in terms of error between fitted model and experimental data. The second part compares the elemental abundances obtained from the different methods. 

#### Resulting Fits
The resulting fits are first visually compared and next the error between the fit and experimental data is shown.

In [19]:
multimodels = quant.get_multimodels()
multimodelspw = quantpw.get_multimodels()

16384it [00:10, 1551.52it/s]
16384it [00:11, 1480.35it/s]
16384it [00:12, 1298.72it/s]
16384it [00:10, 1517.68it/s]
16384it [00:12, 1332.25it/s]
16384it [00:08, 2004.58it/s]
16384it [00:07, 2082.30it/s]
16384it [00:07, 2109.07it/s]
16384it [00:07, 2074.78it/s]
16384it [00:08, 2016.88it/s]


In [38]:
#comparison between experimental data and linear model
em.MultiSpectrumVisualizer([hl]+multimodels)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [20]:
#comparison between experimental data and power-law model
em.MultiSpectrumVisualizer([hl]+multimodelspw)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [21]:
#comparison of the total fit between two methods
em.MultiSpectrumVisualizer([hl]+[multimodels[-1]]+[multimodelspw[-1]], labels=['Data','Linear','Powerlaw'])

The error of each fitted spectrum. Check the colorbar indicating the absolute error which is larger for the **Power-law Method**. This shows that the power-law is not a valid model for the backgroud. The linear background is also not perfect and still some bias is present but it is better at approximating the background which in this case is the convolution of a power-law with the low loss signal.

In [55]:
fig, ax = plt.subplots(1,2)
im= ax[0].imshow(quant.fitter.error_matrix)
divider = make_axes_locatable(ax[0])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

im = ax[1].imshow(quantpw.fitter.error_matrix)
divider = make_axes_locatable(ax[1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

#### Elemental maps: visual inspection
A visual inspection to compare the different methods with the ground truth. Note that a quantitative comparison from these images is difficult to do. However some artifacts can already be seen. For example, the oxgyen map for conventional shows a negative content in the left top square. Also for the power-law method, the carbon content is not homogeneous over the map.

In [45]:
mths = [maps, int_maps, quantpw.elemental_maps, quant.elemental_maps]
mths_nms = ['Ground truth', 'Conventional', 'Power-law Method', 'Linear Method']

fig, ax = plt.subplots(4, len(elements))
for jj in range(len(mths)):
    for ii in range(len(elements)):
        if ii == 0:
            vmin = -0.1
            vmax = 1.5*mths[jj][ii].max()
        
        ax[jj,ii].imshow(mths[jj][ii], vmin=vmin, vmax=vmax)

        if ii == 0:
            ax[jj,ii].set_ylabel(mths_nms[jj])
        if jj == 0:
            ax[jj,ii].set_title(elements[ii])


#### Elemental abundance: quantitative comparison
Since we know the ground truth of the different regions we can compare the average values to see if there is any bias (accuracy) present. Moreover because the different regions have a constant abundance, the standard devitiation can be used to have some measure of precision. One could identify 4 different regions on the sample. 

In [28]:
def get_average_std(mask, mths, mths_nms, rn=2):
    """
    Small function to get the average and standard deviation from the region 
    identified by mask. 
    """
    for ii in range(maps.shape[0]):
            print('Mask, element: '+str(elements[ii]))
            theo = maps[ii][mask.astype('bool')].flatten().mean()
            print('Theoretical value: ' + str(theo))
            
            for kk in range(len(mths)):
                ndata = mths[kk][ii][mask.astype('bool')].flatten()
                avg = ndata.mean()
                std = ndata.std()
                print(mths_nms[kk] + ' method: '+str(np.round(avg,rn))+' +- ' + str(np.round(std, rn)))
                
            print(' ')

##### Region 0 

In [ ]:
fig, ax = plt.subplots()
ax.imshow(mask0)

get_average_std(mask0, mths, mths_nms)

##### Region 1 

In [46]:
fig, ax = plt.subplots()
ax.imshow(mask1)

get_average_std(mask1, mths, mths_nms)

Mask, element: C
Theoretical value: 1.0
Ground truth method: 1.0 +- 0.0
Conventional method: 0.93 +- 0.09
Power-law Method method: 1.23 +- 0.04
Linear Method method: 1.04 +- 0.07
 
Mask, element: N
Theoretical value: 1.0
Ground truth method: 1.0 +- 0.0
Conventional method: 0.97 +- 0.21
Power-law Method method: 0.81 +- 0.05
Linear Method method: 1.0 +- 0.08
 
Mask, element: O
Theoretical value: 0.0
Ground truth method: 0.0 +- 0.0
Conventional method: -0.12 +- 0.18
Power-law Method method: 0.0 +- 0.0
Linear Method method: -0.05 +- 0.07
 
Mask, element: Fe
Theoretical value: 0.0
Ground truth method: 0.0 +- 0.0
Conventional method: -0.0 +- 0.06
Power-law Method method: 0.0 +- 0.0
Linear Method method: -0.02 +- 0.02
 


##### Region 2 

In [31]:
fig, ax = plt.subplots()
ax.imshow(mask2)

get_average_std(mask2, mths, mths_nms)

Mask, element: C
Theoretical value: 1.0
Ground truth method: 1.0 +- 0.0
Conventional method: 0.93 +- 0.12
Power-law Method method: 0.97 +- 0.06
Linear Method method: 1.04 +- 0.09
 
Mask, element: N
Theoretical value: 0.0
Ground truth method: 0.0 +- 0.0
Conventional method: -0.16 +- 0.25
Power-law Method method: 0.0 +- 0.0
Linear Method method: -0.01 +- 0.1
 
Mask, element: O
Theoretical value: 1.0
Ground truth method: 1.0 +- 0.0
Conventional method: 1.02 +- 0.2
Power-law Method method: 0.52 +- 0.07
Linear Method method: 0.96 +- 0.09
 
Mask, element: Fe
Theoretical value: 0.5
Ground truth method: 0.5 +- 0.0
Conventional method: 0.5 +- 0.07
Power-law Method method: 0.34 +- 0.02
Linear Method method: 0.49 +- 0.03
 


##### Region 3

In [36]:
fig, ax = plt.subplots()
ax.imshow(mask3)

get_average_std(mask3, mths, mths_nms)

Mask, element: C
Theoretical value: 1.0
Ground truth method: 1.0 +- 0.0
Conventional method: 0.93 +- 0.12
Power-law Method method: 1.05 +- 0.05
Linear Method method: 1.04 +- 0.11
 
Mask, element: N
Theoretical value: 0.0
Ground truth method: 0.0 +- 0.0
Conventional method: -0.23 +- 0.27
Power-law Method method: 0.0 +- 0.0
Linear Method method: -0.02 +- 0.11
 
Mask, element: O
Theoretical value: 0.0
Ground truth method: 0.0 +- 0.0
Conventional method: -0.02 +- 0.21
Power-law Method method: 0.0 +- 0.0
Linear Method method: -0.05 +- 0.09
 
Mask, element: Fe
Theoretical value: 1.0
Ground truth method: 1.0 +- 0.0
Conventional method: 1.03 +- 0.07
Power-law Method method: 0.82 +- 0.02
Linear Method method: 0.99 +- 0.03
 
